In [1]:
# Run this first!!!
from IPython.display import display, HTML

from __future__ import division
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
import util
sys.path.append('../aho_merging')
from failure_chain_mc import FailureChain
sys.path.append('../common')
from sampling_misc import sample_stat_failure_chain_dist

## Fixing the Markov Chain

## Does there seem to be a Stationary Distribution?

To check whether there is a stationary distribution. I will look at the distribution after the chain has ran for a long time and collect data. I will do this two times (but vary how long we let the chain run for) and see if the two supposed stationary distributions are the same (we know if one exists it must be unique).

In [18]:
def compare_stat_dists(depth1, depth2, num_samples, probs):
    fc = FailureChain(probs)
    stat1 = fc.get_stationary_dist(depth1, num_samples)
    print 'Distribution after %d runs:\t' % depth1,
    print stat1
    stat2 = fc.get_stationary_dist(depth2, num_samples)
    print 'Distribution after %d runs:\t' % depth2,
    print stat2
    print 'Relative Difference From 1st:\t'
    print [(stat1[i] - stat2[i])/stat1[i] 
           if i < len(stat1) and i < len(stat2) else float('inf')
           for i in range(max(len(stat1), len(stat2)))]

In [19]:
DEPTH1 = 100
DEPTH2 = 200
NUM_SAMPLES = 10000
PROBS = [0.25, 0.5, 0.75, 0.3]

compare_stat_dists(DEPTH1, DEPTH2, NUM_SAMPLES, PROBS)

Distribution after 100 runs:	[0.1876, 0.3856, 0.2996, 0.1032, 0.0213, 0.0025, 0.0002]
Distribution after 200 runs:	[0.1809, 0.3932, 0.2976, 0.1058, 0.0194, 0.0028, 0.0001, 0.0002]
Relative Difference From 1st:	
[0.03571428571428563, -0.01970954356846472, 0.006675567423230981, -0.02519379844961245, 0.08920187793427226, -0.11999999999999997, 0.5, inf]


Looks pretty close to me! I am thinking that there is one. We basically have a finite state space here because it is really unlikely that we keep going. The one case where this is not true is if we have a letter that will always appear in a set. In this case there will be no stationary distribution since we go off to infinity.

## Comparing Model against Truth

In [4]:
def compare_stat_dists_against_real(num_samples, probs, depth):
    fc = FailureChain(probs)
    theoretical = fc.get_stationary_dist(depth, num_samples)
    print 'Theoretical: \t', theoretical
    sampled = sample_stat_failure_chain_dist(num_samples, probs, depth)
    print 'Sampled: \t', sampled
    s_len, t_len = len(sampled), len(theoretical)
    print 'Relative Difference: '
    print [(sampled[i] - theoretical[i]) / theoretical[i] 
           if i < s_len and i < t_len else float('inf')
           for i in range(max(s_len, t_len))]

In [11]:
NUM_SAMPLES = 1000
PROBS = [0.25, 0.5, 0.75, 0.3]
DEPTH = 50

compare_stat_dists_against_real(NUM_SAMPLES, PROBS, DEPTH)

Theoretical: 	[0.195, 0.364, 0.321, 0.103, 0.014, 0.003]
Sampled: 	[0.262, 0.319, 0.21, 0.096, 0.064, 0.033, 0.007, 0.006, 0.002, 0.0, 0.001]
Relative Difference: 
[0.3435897435897436, -0.12362637362637359, -0.3457943925233645, -0.06796116504854362, 3.5714285714285716, 10.0, inf, inf, inf, inf, inf]
